Après avoir tenter d'essayer d'identifier les entreprises dans les textes originaux, nous nous sommes confronté à de nombreux problèmes. Suite à cela, et avec l'accord de notre encadrant Stat_app, nous avons fait le choix d'introduire le nom de certaines entreprises dans les articles originaux. Ce notebook vise à appliquer cette démarche

In [1]:
import pandas as pd

In [2]:
# Charger à partir du fichier pickle
data = pd.read_pickle('data.pkl')
data.head(5)

,Article,Date,Auteur,Nombre de mots,Journal,Titre,ID
0,\nMetropolitan Desk; SECTMB\nCan an Ambitious ...,31 December 2023,Nick Tabor,529,New York Times,Copyright 2023 The New York Times Company. Al...,NYTF000020240104ejcv0000d
1,\n\nMagazine Desk; SECTMM\nWhen Jim Brown and ...,31 December 2023,Wesley Morris,422,New York Times,"When Jim Brown and Raquel Welch, Two Sexy Star...",NYTF000020231231ejcv0006h
2,\n\nMagazine Desk; SECTMK\nTalking During Movi...,31 December 2023,None,179,New York Times,Talking During Movies: Totally Evil or Part of...,NYTF000020231231ejcv00064
3,\n\nMagazine Desk; SECTMK\nLet Kids Vote!\n\n4...,31 December 2023,None,454,New York Times,Let Kids Vote!,NYTF000020231231ejcv00063
4,\n\nMagazine Desk; SECTMK\nAre We Doomed to Di...,31 December 2023,Christina Caron,428,New York Times,Are We Doomed to Disagree?,NYTF000020231231ejcv0005z


Maintenant que nous avons les informations sur le texte (Date, Auteur, Nombre de mots etc...) Nous pouvons nous permettre de ne garder uniquement le coeur de l'article :

In [3]:
data.insert(1, 'Copy_Article', data['Article'])
data.head(5)

,Article,Copy_Article,Date,Auteur,Nombre de mots,Journal,Titre,ID
0,\nMetropolitan Desk; SECTMB\nCan an Ambitious ...,\nMetropolitan Desk; SECTMB\nCan an Ambitious ...,31 December 2023,Nick Tabor,529,New York Times,Copyright 2023 The New York Times Company. Al...,NYTF000020240104ejcv0000d
1,\n\nMagazine Desk; SECTMM\nWhen Jim Brown and ...,\n\nMagazine Desk; SECTMM\nWhen Jim Brown and ...,31 December 2023,Wesley Morris,422,New York Times,"When Jim Brown and Raquel Welch, Two Sexy Star...",NYTF000020231231ejcv0006h
2,\n\nMagazine Desk; SECTMK\nTalking During Movi...,\n\nMagazine Desk; SECTMK\nTalking During Movi...,31 December 2023,None,179,New York Times,Talking During Movies: Totally Evil or Part of...,NYTF000020231231ejcv00064
3,\n\nMagazine Desk; SECTMK\nLet Kids Vote!\n\n4...,\n\nMagazine Desk; SECTMK\nLet Kids Vote!\n\n4...,31 December 2023,None,454,New York Times,Let Kids Vote!,NYTF000020231231ejcv00063
4,\n\nMagazine Desk; SECTMK\nAre We Doomed to Di...,\n\nMagazine Desk; SECTMK\nAre We Doomed to Di...,31 December 2023,Christina Caron,428,New York Times,Are We Doomed to Disagree?,NYTF000020231231ejcv0005z


Dans un premier temps, on supprime l'ID de l'article, qui se situe à la fin du texte

In [4]:
import re

# Fonction pour supprimer le texte après le motif spécifié
def supprimer_texte_apres_motif(article, motifs):
    motif = "|".join(motifs)  # Concaténer les motifs en une seule chaîne de caractères
    match = re.search(motif, article)
    if match:
        return article[:match.start()]
    else:
        return article

# Appliquer la fonction supprimer_texte_apres_motif à la colonne 'Coeur_Article' avec une liste de motifs
data['Copy_Article'] = data['Copy_Article'].apply(lambda x: supprimer_texte_apres_motif(x, ["Document J\d+", "Document NYTF\d+"]))

Vérification :

In [5]:
for i in range(1,5):
    print("Derniers caractères du", f"text_{i}", "avant suppression\n\n", data['Article'][i][-50:-1])
    print("Derniers caractères du", f"text_{i}", "après suppression\n\n", data['Copy_Article'][i][-50:-1])
    print("-----------------------------------------------------------------------------")

Derniers caractères du text_1 avant suppression

 M21, MM22. 

Document NYTF000020231231ejcv0006h


Derniers caractères du text_1 après suppression

 cle appeared in print on page MM20, MM21, MM22. 

-----------------------------------------------------------------------------
Derniers caractères du text_2 avant suppression

 K4, MK5. 

Document NYTF000020231231ejcv00064
 


Derniers caractères du text_2 après suppression

 his article appeared in print on page MK4, MK5. 

-----------------------------------------------------------------------------
Derniers caractères du text_3 avant suppression

 age MK3. 

Document NYTF000020231231ejcv00063
 


Derniers caractères du text_3 après suppression

 o.

This article appeared in print on page MK3. 

-----------------------------------------------------------------------------
Derniers caractères du text_4 avant suppression

 ge MK11. 

Document NYTF000020231231ejcv0005z
 


Derniers caractères du text_4 après suppression

 Š

This article 

On supprime maintenant tout ce qui est placé avant "All Rights Reserved.", qui correspond à la partie d'information du texte (Auteur etc...)

In [6]:
# Fonction pour supprimer le texte après le motif spécifié
def supprimer_texte_avant_motif(article, motif):
    match = re.search(motif, article)
    if match:
        return article[match.end():]
    else:
        return article

# Appliquer la fonction supprimer_texte_apres_motif à la colonne 'Coeur_Article' avec une liste de motifs
data['Copy_Article'] = data['Copy_Article'].apply(lambda x: supprimer_texte_avant_motif(x, "All Rights Reserved."))

Vérification :

In [7]:
for i in range(1,5):
    print("Derniers caractères du", f"text_{i}", "avant suppression\n\n", data['Article'][i][0:100],"\n\n\n")
    print("Derniers caractères du", f"text_{i}", "après suppression\n\n", data['Copy_Article'][i][0:100])
    print("-----------------------------------------------------------------------------")

Derniers caractères du text_1 avant suppression

 

Magazine Desk; SECTMM
When Jim Brown and Raquel Welch, Two Sexy Stars, Crossed Paths

By Wesley Mo 



Derniers caractères du text_1 après suppression

  

In their one movie together, their chemistry was radical.

Jim Brown & Raquel Welch B. 1936 and 1
-----------------------------------------------------------------------------
Derniers caractères du text_2 avant suppression

 

Magazine Desk; SECTMK
Talking During Movies: Totally Evil or Part of the Fun?

179 words
31 Decemb 



Derniers caractères du text_2 après suppression

  

debatethis

Talking during movies: Totally evil or part of the fun?

show of hands

The biggest p
-----------------------------------------------------------------------------
Derniers caractères du text_3 avant suppression

 

Magazine Desk; SECTMK
Let Kids Vote!

454 words
31 December 2023
The New York Times
NYTF
Late Edit 



Derniers caractères du text_3 après suppression

  

LET KIDS

VOTE!

by Kath

In [8]:
data.rename(columns={'Copy_Article': 'Coeur_Article'}, inplace=True)
data.head(5)

,Article,Coeur_Article,Date,Auteur,Nombre de mots,Journal,Titre,ID
0,\nMetropolitan Desk; SECTMB\nCan an Ambitious ...,"\n\nStony Brook University, one of two state ...",31 December 2023,Nick Tabor,529,New York Times,Copyright 2023 The New York Times Company. Al...,NYTF000020240104ejcv0000d
1,\n\nMagazine Desk; SECTMM\nWhen Jim Brown and ...,"\n\nIn their one movie together, their chemis...",31 December 2023,Wesley Morris,422,New York Times,"When Jim Brown and Raquel Welch, Two Sexy Star...",NYTF000020231231ejcv0006h
2,\n\nMagazine Desk; SECTMK\nTalking During Movi...,\n\ndebatethis\n\nTalking during movies: Tota...,31 December 2023,None,179,New York Times,Talking During Movies: Totally Evil or Part of...,NYTF000020231231ejcv00064
3,\n\nMagazine Desk; SECTMK\nLet Kids Vote!\n\n4...,\n\nLET KIDS\n\nVOTE!\n\nby Katherine Cusuman...,31 December 2023,None,454,New York Times,Let Kids Vote!,NYTF000020231231ejcv00063
4,\n\nMagazine Desk; SECTMK\nAre We Doomed to Di...,\n\nare we DOOMED TO DISAGREE?\n\nwhy it's so...,31 December 2023,Christina Caron,428,New York Times,Are We Doomed to Disagree?,NYTF000020231231ejcv0005z


On cherche maintenant à insérer de manière aléatoire une phrase dans le texte. Ainsi, on pourra mettre dans cette phrase le nom d'une entreprise

In [9]:
import random

def insertion_phrase_dans_article(phrase, article):
    # Trouver tous les emplacements des points dans l'article
    emplacements_points = [i for i, char in enumerate(article) if char == '.']

    # Vérifier s'il y a des points dans l'article
    if emplacements_points:
        # Choisir aléatoirement l'un des emplacements des points
        indice_insertion = random.choice(emplacements_points)
        # Insérer la phrase juste après le point choisi
        article = article[:indice_insertion+1] + " " + phrase + article[indice_insertion+1:]
    else:
        # S'il n'y a pas de point, insérer la phrase au début de l'article
        article = phrase + " " + article
    return article

phrase_a_inserer = "AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA."
article_insertion = data['Coeur_Article'][0]
article_insertion = insertion_phrase_dans_article(phrase_a_inserer, article_insertion)

# Afficher l'article avec la phrase insérée
print(article_insertion)

 

Stony Brook University, one of two state schools designated as flagships, has aggressively fund-raised and recruited students.

Shortly after she became governor of New York, Kathy Hochul made an announcement that caught the higher-education world off guard: The state, which has generally regarded all its public universities as equals, would start treating two universities as ''flagships. AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA.''

More surprising still, her choices for the elite label didn't include Binghamton University, which has often been regarded as New York's top academic public university. Instead, the governor said the distinction would go to Stony Brook University, on Long Island, and the University at Buffalo, in her hometown.

For leaders at Stony Brook, the promotion wasn't at all a surprise. It was the next step in a yearslong campaign to transform the institution from a well